# Working with sequence data

We need some data to provide an example of how to do this. Here is a very simple dataset. The thing to note is that we have a list of text -- each item in the list is a observation/thing to cluster -- and within the text we have a series of tokens delimited by something -- in this case commas. We are going to look at a few ways of turning data that looks like this into data that can be clustered, visualised, and otherwise played with using data science tools.

In [1]:
example_data = ["item one, item two, another item of words, some tokens, and so on, item one, item two",
                "item two, item three, and so on, and another, and more, item two",
                "item one, another item of words, some tokens, item one, and more",
                "item two, item one, some tokens, and so on, and another",
                "item three, item two, item one, and so on, and more, and so on",
                "item four, item one, item two, item three, some tokens, and so on, item four",
                "item one, item two, item three",
                "some tokens, another item of words, item four"
               ]

The thing we really need is sklearn's feature processing tools, primarily from their text processing suite. We will therefore load that up.

In [2]:
import sklearn.feature_extraction.text

## The Basic Approach

The most naive approach (but also the simplest to implement) is to assume that order doesn't matter within each observation. This is known as a "Bag of Words" model, since it views a text document not as an ordered sequence of words, but as a big unordered "bag" (or multiset) of words. The relevant tool in sklearn is the [``CountVectorizer``](http://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage). The ``CountVectorizer`` takes a lot of parameters, but most of the defaults are just fine. In our case the tokens in each observation are comma delimited, which differs from the default of space delimited, so we'll have to pass that in.

The tokenizer just needs to take a string and return a list of tokens. We will be naive about this, but you can write a more complex tokenizer suitable for your input.

In [3]:
def comma_tokenizer(instring):
    return [x.strip() for x in instring.split(',')]

Now we can pass this to the ``CountVectorizer`` and then ``fit_transform`` our data to get a nice mathematical representation out that is suitable for further processing.

In [4]:
data = sklearn.feature_extraction.text.CountVectorizer(tokenizer=comma_tokenizer).fit_transform(example_data)
data

<8x9 sparse matrix of type '<class 'numpy.int64'>'
	with 36 stored elements in Compressed Sparse Row format>

As you can see we get some sort of "sparse matrix object", which may not seem like what you want, but actually is. The catch is that the data is potentially *very* high dimensional, and a sparse matrix is the right way to store that without using up all the memory on your computer. Since we actually have a very small dataset we can turn it into data that we can look at by calling ``toarray``. Don't do this unless it is a similarly small dataset.

In [5]:
data.toarray()

array([[0, 0, 1, 1, 0, 2, 0, 2, 1],
       [1, 1, 1, 0, 0, 0, 1, 2, 0],
       [0, 1, 0, 1, 0, 2, 0, 0, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 1],
       [0, 1, 2, 0, 0, 1, 1, 1, 0],
       [0, 0, 1, 0, 2, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 1]], dtype=int64)

This probably looks more like the sort math representation one might have expected. We can now run clustering or dimension reduction on this. The catch is that the sparse version is not an acceptable input for many algorithms. You will likely be better off using dimension reduction first -- but very few dimension reduction algorithms support potentially very high dimensional sparse input. The only practicable options that I know of are ``TruncatedSVD`` and ``UMAP``\*. For now we'll just use ``TruncatedSVD``. We can use it to reduce this to a (dense) 3-dimensional representation. If you actaully have real data you will probably want to reduce to something more like 100-dimensional.

\* If you want to use UMAP then you can ``pip install umap-learn``, do a ``import umap`` and then simply replace ``sklearn.decomposition.TruncatedSVD`` with ``umap.UMAP``.

In [6]:
import sklearn.decomposition

In [7]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=3).fit_transform(data)
reduced_data

array([[ 3.02002865, -0.71724343, -0.6719972 ],
       [ 2.07058085,  1.69309447, -0.06847205],
       [ 1.68397559, -1.62050723, -0.84118247],
       [ 1.90734447, -0.00639448, -0.20584057],
       [ 2.35201181,  1.01715398, -0.24136204],
       [ 2.35355067, -0.27500879,  1.78254853],
       [ 1.31702764,  0.22477508, -0.1577074 ],
       [ 0.61129028, -1.04639031,  0.91681553]])

Now we can throw this directly to clustering if we wish. You can see [this notebook](https://github.com/vpoulin/Notebooks/blob/master/Statistical_Techniques/Machine_Learning/(U%29%20Clustering.ipynb) for an example. If you want to visualize consider [this notebook](https://github.com/vpoulin/Notebooks/blob/master/Data_Exploration_and_Visualization/Visualizing%20Data%20with%20UMAP.ipynb) instead.

## What if Order Matters?

If the bag of words model is too simplistic we can go a step further and take some order into account. The next simplest approach is to use ``n-grams``, which are simply sequences of up to *n* tokens that all occur together in the document. The upside is that token order now matters (as we are running a sliding window over the sequence), the downside is that as *n* gets larger the result will be even more high dimensional, and even sparser. This means that in practice you don't want *n* to get much larger than 2 or 3. Still, we can do this easily, still using the ``CountVectorizer`` -- we just need to pass the range of sizes of ``n-grams`` we will allow as the ``ngram_range``. In this example we'll consider 2-grams and 3-grams.

In [8]:
data = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=comma_tokenizer,
    ngram_range=[2,3]
).fit_transform(example_data)
data

<8x49 sparse matrix of type '<class 'numpy.int64'>'
	with 59 stored elements in Compressed Sparse Row format>

Note that the resulting matrix now has 49 columns instead of the 9 we had earlier. We can still use ``TruncatedSVD`` to cut this down to something smaller.

In [9]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=5).fit_transform(data)
reduced_data

array([[ 2.96454445e+00, -1.66792223e+00,  4.57151097e-01,
         8.81168298e-01, -5.59430963e-01],
       [ 5.45935647e-01,  2.08338423e+00, -2.63834756e-01,
         1.94452683e+00, -1.83222597e-02],
       [ 3.13786598e-01, -5.30601275e-01,  1.95448571e-01,
         6.84316296e-01,  2.43585599e+00],
       [ 6.35785252e-01,  7.45545908e-01,  9.29169277e-01,
         4.48915090e-01, -4.98937112e-01],
       [ 8.53672852e-02,  6.52010515e-01,  2.74104098e+00,
        -6.30199822e-01,  2.23772227e-01],
       [ 2.41082610e+00,  1.27611889e+00, -7.61045168e-01,
        -1.61744427e+00,  4.68569407e-01],
       [ 1.01932365e+00,  3.60595227e-01, -2.57559893e-01,
        -2.16630585e-01,  7.12202068e-02],
       [-3.77776040e-18, -6.12486293e-16, -6.81471305e-15,
         4.23490260e-15,  7.00804977e-15]])

## What if Token Frequency Matters?

Sometimes some tokens are more important than others. For example in a bunch of english language documents the count of the word "the" is probably far less important in characterising a document than the count of a more specialised word that happens to occur surprisingly frequently within that document. Fortunately we can handle this with TF-IDF, which stands for "Term Frequency - Inverse Document Frequency". In other words we are going to normalize the token counts by how frequently the tokens occur across observations -- tokens that show up everywhere are going to have their counts reduced, while tokens that show up in very few documents will be deemed more important. This can all be handled automatically by the ``TfidfVectorizer``.

In [10]:
data = sklearn.feature_extraction.text.TfidfVectorizer(tokenizer=comma_tokenizer).fit_transform(example_data)
data

<8x9 sparse matrix of type '<class 'numpy.float64'>'
	with 36 stored elements in Compressed Sparse Row format>

Again we can reduce the sparse matrix to a dense lower dimensional representation with ``TruncatedSVD``.

In [11]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=3).fit_transform(data)
reduced_data

array([[ 0.85491952,  0.13144172,  0.32398167],
       [ 0.72924668, -0.48233062, -0.21631292],
       [ 0.63994254,  0.34461524,  0.59895244],
       [ 0.77128025, -0.12779085,  0.07010208],
       [ 0.78867917, -0.36752165, -0.06472442],
       [ 0.67525806,  0.40506237, -0.56810351],
       [ 0.72470911, -0.2888536 , -0.06848349],
       [ 0.39644418,  0.8653536 , -0.1823772 ]])

We can also play the same games with ``n-grams`` as before, since ``TfidfVectorizer`` supports an ``ngram_range`` argument.

In [12]:
data = sklearn.feature_extraction.text.TfidfVectorizer(
    tokenizer=comma_tokenizer,
    ngram_range=[2,3]
).fit_transform(example_data)
data

<8x49 sparse matrix of type '<class 'numpy.float64'>'
	with 59 stored elements in Compressed Sparse Row format>

In [13]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=5).fit_transform(data)
reduced_data

array([[ 5.42194569e-01, -1.14704167e-01,  4.20145022e-01,
         0.00000000e+00, -8.55798991e-02],
       [ 2.81586661e-01,  3.92055321e-01, -3.63892358e-01,
         1.97114101e-14,  6.70999907e-01],
       [ 7.37237598e-02, -8.34303675e-02,  7.94948510e-01,
         2.23779626e-14,  4.57076070e-01],
       [ 1.93893924e-01,  7.28579860e-01,  1.07034677e-01,
         1.69194249e-15,  8.54769743e-02],
       [ 3.05247537e-02,  6.13561048e-01,  2.34476832e-01,
        -1.45739649e-14, -5.28568786e-01],
       [ 7.49454466e-01, -1.01496487e-01, -1.36740143e-01,
        -6.29187083e-15, -1.62056657e-01],
       [ 8.01417083e-01, -1.57201271e-01, -1.36470660e-01,
        -3.21855853e-15, -6.89104124e-02],
       [ 2.11430114e-16,  7.03776096e-16, -8.67992610e-15,
         1.00000000e+00, -3.25441824e-14]])

Again you are ready for [clustering](https://github.com/vpoulin/Notebooks/blob/master/Statistical_Techniques/Machine_Learning/(U%29%20Clustering.ipynb) or [visualization](https://github.com/vpoulin/Notebooks/blob/master/Data_Exploration_and_Visualization/Visualizing%20Data%20with%20UMAP.ipynb).

## Still More Complex?

We can get a little more advanced again. The first step to that is to use skipgrams -- this counts co-occurences within windows. It has many of the benefits of large ``n-gram``, but avoids increasing the dimension and sparsity anywhere near as much. This takes a little more work, but we can put one together fairly quickly. Here is a simple example [taken from stackoverflow](https://stackoverflow.com/questions/39725052/is-there-any-way-implementing-skip-gram-with-scikit-learn):

In [14]:
from toolz import itertoolz, compose
from toolz.curried import map as cmap, sliding_window, pluck

class SkipGramVectorizer(sklearn.feature_extraction.text.CountVectorizer):
    def build_analyzer(self):    
        preprocess = self.build_preprocessor()
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()
        return lambda doc: self._word_skip_grams(
                compose(tokenize, preprocess, self.decode)(doc),
                stop_words)

    def _word_skip_grams(self, tokens, stop_words=None):
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        return compose(cmap(' '.join), pluck([0, 2]), sliding_window(3))(tokens)


In [15]:
data = SkipGramVectorizer(tokenizer=comma_tokenizer).fit_transform(example_data)
data

<8x21 sparse matrix of type '<class 'numpy.int64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [16]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=5).fit_transform(data)
reduced_data

array([[ 1.56971417e+00, -3.01811904e-14,  1.51733213e+00,
        -1.16067590e-15,  0.00000000e+00],
       [-6.03705293e-16,  1.58113883e+00,  2.76759192e-14,
         4.11539167e-01,  1.15353176e+00],
       [ 1.09024347e-16,  7.66175736e-18,  1.02147694e-16,
         1.63134026e+00, -5.82004271e-01],
       [ 9.05112429e-01, -3.62298979e-15,  1.85787843e-01,
         2.98175853e-15, -1.24018790e-15],
       [-8.26982847e-16,  1.58113883e+00,  2.69594492e-14,
        -4.11539167e-01, -1.15353176e+00],
       [ 1.80071984e+00,  2.63253189e-14, -1.27969904e+00,
        -5.06191996e-16,  2.03201958e-16],
       [ 3.14603595e-01,  8.09668973e-15, -3.90264441e-01,
        -4.87365390e-16,  2.66108648e-15],
       [ 3.14603595e-01,  7.83378515e-15, -3.90264441e-01,
        -1.55665665e-16, -1.91182326e-15]])

Again you are ready for [clustering](https://github.com/vpoulin/Notebooks/blob/master/Statistical_Techniques/Machine_Learning/(U%29%20Clustering.ipynb) or [visualization](https://github.com/vpoulin/Notebooks/blob/master/Data_Exploration_and_Visualization/Visualizing%20Data%20with%20UMAP.ipynb).

We can go a step further and do a more principled version of TF-IDF by looking at the pointwise mutual information (essentially the log likelihood ratio).

In [17]:
import numpy as np
import scipy.sparse

In [18]:
row_sums = np.array(data.sum(axis=1).T)[0]
col_sums = np.array(data.sum(axis=0))[0]
row_diag = scipy.sparse.spdiags(1./row_sums, 0, row_sums.shape[0], row_sums.shape[0])
col_diag = scipy.sparse.spdiags(1./col_sums, 0, col_sums.shape[0], col_sums.shape[0])

In [19]:
pmi_matrix = row_diag * data * col_diag
pmi_matrix.data = np.log(pmi_matrix.data)

In [20]:
pmi_matrix.toarray()

array([[ 0.        ,  0.        ,  0.        , -1.60943791, -1.60943791,
         0.        ,  0.        ,  0.        ,  0.        , -1.60943791,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -2.7080502 ,  0.        ,  0.        ,  0.        ,
        -1.60943791],
       [-1.38629436, -1.38629436,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -1.38629436,  0.        ,  0.        ,
        -2.07944154,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.09861229,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -1.09861229,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -1.09861229,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        

In [21]:
reduced_data = sklearn.decomposition.TruncatedSVD(n_components=5).fit_transform(pmi_matrix)
reduced_data

array([[ 3.11237955e+00,  1.11809651e-14,  2.75106555e+00,
        -8.25790982e-16, -6.62815062e-01],
       [ 2.51457783e-16,  2.68454749e+00, -1.04879723e-14,
        -1.69785691e+00,  6.96105145e-16],
       [ 1.80231766e-16,  3.71119795e-16,  3.91986526e-16,
        -1.11080477e-15, -1.12822827e-15],
       [ 1.82833742e+00,  3.10721863e-15,  5.51003248e-01,
         1.31619490e-15,  1.89573709e+00],
       [ 1.71497672e-17,  2.68454749e+00, -9.49384678e-15,
         1.69785691e+00, -1.09426385e-15],
       [ 4.25233486e+00, -9.49821904e-15, -2.22098479e+00,
         1.17113371e-16, -3.15774647e-01],
       [ 2.20860278e-01, -1.37273469e-15, -2.83910146e-01,
         3.02102991e-18, -1.36591980e-01],
       [ 2.20860278e-01, -1.24956150e-15, -2.83910146e-01,
        -7.48805394e-17, -1.36591980e-01]])

Again you are ready for [clustering](https://github.com/vpoulin/Notebooks/blob/master/Statistical_Techniques/Machine_Learning/(U%29%20Clustering.ipynb) or [visualization](https://github.com/vpoulin/Notebooks/blob/master/Data_Exploration_and_Visualization/Visualizing%20Data%20with%20UMAP.ipynb).